## Softmax

$$softmax(x)_i =  \frac{\exp{x_i}}{\sum_{j=1}^{n} \exp{x_j}}$$

Exponential

In [4]:
exp(32)

7.896296018268069e13

Summation

In [5]:
test = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [6]:
sum(test)

6

In [10]:
den_softmax(x) = exp.(x)./sum(exp.(x))

den_softmax (generic function with 1 method)

In [11]:
den_softmax([0.1, 0.2])

2-element Array{Float64,1}:
 0.4750208125210601
 0.52497918747894  

In [21]:
using Pkg;
Pkg.add("Knet")
Pkg.build("SpecialFunctions")
Pkg.build("CodecZlib")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/fvheQ/deps/build.log`
  Building CodecZlib → `~/.julia/packages/CodecZlib/DAjXH/deps/build.log`


In [5]:
using Knet

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1192


In [24]:
softmax([0.1, 0.2])

2-element Array{Float64,1}:
 0.47502081252106
 0.52497918747894

In [28]:
den_softmax([0.000000000000000000000000001, 0.000000000000000000000000000000002])

2-element Array{Float64,1}:
 0.5
 0.5

In [29]:
softmax([0.000000000000000000000000001, 0.000000000000000000000000000000002])

2-element Array{Float64,1}:
 0.5
 0.5

Housing example to illustrate gradient descent
https://denizyuret.github.io/Knet.jl/latest/backprop.html#Stochastic-Gradient-Descent-1

In [6]:
include(Knet.dir("data","housing.jl"))

In [7]:
x,y = housing()  # x is (13,506); y is (1,506)


┌ Info: Downloading https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data to /home/yizhe/.julia/packages/Knet/T1oum/data/housing/housing.data
└ @ Main /home/yizhe/.julia/packages/Knet/T1oum/data/housing.jl:27
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49082  100 49082    0     0  67143      0 --:--:-- --:--:-- --:--:-- 67051


([-0.419367 -0.416927 … -0.407361 -0.41459; 0.284548 -0.48724 … -0.48724 -0.48724; … ; 0.440616 0.440616 … 0.402826 0.440616; -1.0745 -0.491953 … -0.864446 -0.668397], [24.0 21.6 … 22.0 11.9], [-0.419367 -0.416927 … -0.407361 -0.41459; 0.284548 -0.48724 … -0.48724 -0.48724; … ; 0.440616 0.440616 … 0.402826 0.440616; -1.0745 -0.491953 … -0.864446 -0.668397], [24.0 21.6 … 22.0 11.9])

# Dataset description https://www.kaggle.com/c/boston-housing

In [32]:
x

13×506 Array{Float64,2}:
 -0.419367  -0.416927  -0.416929  …  -0.413038  -0.407361  -0.41459 
  0.284548  -0.48724   -0.48724      -0.48724   -0.48724   -0.48724 
 -1.28664   -0.592794  -0.592794      0.115624   0.115624   0.115624
 -0.272329  -0.272329  -0.272329     -0.272329  -0.272329  -0.272329
 -0.144075  -0.73953   -0.73953       0.157968   0.157968   0.157968
  0.413263   0.194082   1.28145   …   0.983986   0.724955  -0.362408
 -0.119895   0.366803  -0.265549      0.796661   0.736268   0.434302
  0.140075   0.556609   0.556609     -0.772919  -0.667776  -0.61264 
 -0.981871  -0.867024  -0.867024     -0.981871  -0.981871  -0.981871
 -0.665949  -0.986353  -0.986353     -0.802418  -0.802418  -0.802418
 -1.45756   -0.302794  -0.302794  …   1.1753     1.1753     1.1753  
  0.440616   0.440616   0.396035      0.440616   0.402826   0.440616
 -1.0745    -0.491953  -1.20753      -0.982076  -0.864446  -0.668397

In [33]:
y

1×506 Array{Float64,2}:
 24.0  21.6  34.7  33.4  36.2  28.7  …  16.8  22.4  20.6  23.9  22.0  11.9

Playing with grad

In [48]:
funny(x) = x^2

funny (generic function with 1 method)

In [49]:
funny(3)

9

In [50]:
grad(funny)(3)

6

In [54]:
d_funny = grad(funny)

(::getfield(AutoGrad, Symbol("#gradfun#8")){getfield(AutoGrad, Symbol("##gradfun#6#7")){typeof(funny),Int64,Bool}}) (generic function with 1 method)

In [55]:
d_funny(3)

6

Continuing with the example

In [56]:
predict(w,x) = w[1]*x .+ w[2]

predict (generic function with 1 method)

In [57]:
loss(w,x,y) = mean(abs2,y-predict(w,x))
lossgradient = grad(loss)	# grad gives the gradient function wrt w
w = [ 0.1*rand(1,13), 0.0 ]	# initialize the weight vector and bias

2-element Array{Any,1}:
  [0.0948516 0.0765842 … 0.0626769 0.0850844]
 0.0                                         

In [58]:
for epoch in 1:10
    dw = lossgradient(w, x, y)
    for i in 1:length(w)
        w[i] -= lr * dw[i]
    end
end

UndefVarError: UndefVarError: lr not defined

## Jacobian and Hessian Matrices

The matrix containing all such partial derivatives is known as a Jacobian matrix. Second derivative, derivative of derivative can be collected together into a matrix called the Hessian matrix. The Hessian matrix is the Jacobian of the gradient. 

The (directional) second derivative tells us how well we can expect a gradient descent step to perform.

The second derivative can be used to determine whether a critical point is a local maximum, a local minimum, or saddle point. The second derivative test is  that when $f(x) = 0$ and $f(x) > 0$, we can conclude that $x$ is a local minimum. Similarly, when $f(x) = 0$ and $f(x) < 0$, we can conclude that $x$ is a local maximum. But when $f(x) = 0$, the test is inconclusive. In this case $x$ may be a saddle point, or a part of a flat region.

Newton's method is a second order optimization algorithm uses the Hessian matrix. (And those only uses the gradient, such as gradient descent is called first order optimization algorithm). 

### Example of Newton Methods for Logistic Regression

Inspired by: [this](https://thelaziestprogrammer.com/sharrington/math-of-machine-learning/solving-logreg-newtons-method). Using [Boston Housing Dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/)

$$
h(x) = \frac{1}{(1 + e^{-z})},\ \ z = \theta_{1}x + \theta_{2}
$$

In [10]:
den_sigmoid(x, θ1, θ2) = 1 ./ (1 .+ exp.(-(θ1 .* x .+ θ2)))

den_sigmoid (generic function with 1 method)

In [11]:
den_sigmoid(x, 15.1, -0.4)

13×506 Array{Float64,2}:
 0.829294  0.829639  0.829639  0.829722  …  0.830188  0.830987  0.829969
 0.907589  0.81947   0.81947   0.81947      0.81947   0.81947   0.81947 
 0.671144  0.803325  0.803325  0.666948     0.892412  0.892412  0.892412
 0.849114  0.849114  0.849114  0.849114     0.849114  0.849114  0.849114
 0.864821  0.779107  0.779107  0.762338     0.896411  0.896411  0.896411
 0.917833  0.899717  0.963787  0.95326   …  0.951845  0.938483  0.837207
 0.867623  0.914261  0.849981  0.766904     0.942495  0.939133  0.919406
 0.894738  0.928016  0.928016  0.95592      0.773307  0.791208  0.80017 
 0.734608  0.756388  0.756388  0.776923     0.734608  0.734608  0.734608
 0.79151   0.733733  0.733733  0.709916     0.768094  0.768094  0.768094
 0.63238   0.845169  0.845169  0.892153  …  0.959894  0.959894  0.959894
 0.919872  0.919872  0.916524  0.918021     0.919872  0.917043  0.919872
 0.716162  0.818772  0.688361  0.654715     0.734568  0.756862  0.791106

Log likelihood function: 

$$
P(y=1|\ x;\theta) = h_{\theta}(x)
P(y=0|\ x;\theta) = 1 - h_{\theta}(x)
$$

Consolidate to: 

$$
P(y|\ x;\theta) = h_{\theta}(x)^{y}(1-h_{\theta}(x))^{1-y}
$$

Likelihood function is: 

$$
\begin{align}
L(\theta) &= \prod_{i=1}^{n}p(y_{i}|x_{i};\theta) \\
          &= \prod_{i=1}^{n}h_{\theta}(x_{i})^{y_{i}}(1-h_{\theta}(x_{i}))^{1-y_{i}}
\end{align}
$$

Log of likelihood function: 

$$
\begin{align}
\ell(\theta) &= log\ L(\theta) \\
             &= \sum_{i=1}^{n}y_{i}\log(h_{\theta}(x_{i})) + (1-y_{i})\log(1-h_{\theta}(x_{i}))
\end{align}
$$

In [21]:
function den_log_likelihood(x, y, θ1, θ2)
    sigmoid_prob = den_sigmoid(x, θ1, θ2)
    sum(y .* log.(sigmoid_prob) .+ (1 .- y) .* log.(1 .- sigmoid_prob))
end

den_log_likelihood (generic function with 1 method)

In [44]:
den_log_likelihood(x, y, 3, 0.4)

21941.13703759911

Gradient: 

$$
\nabla \ell = \left\langle\matrix{
\sum_{i=1}^{n}(y_{i} - h_{\theta}(x_{i}))x_{i}\cr
\sum_{i=1}^{n}(y_{i} - h_{\theta}(x_{i}))
}\right\rangle
$$

In [51]:
function den_gradient(x, y, θ1, θ2)
    sigmoid_prob = den_sigmoid(x, θ1, θ2)
    [sum((y .- sigmoid_prob) .* x) sum((y .- sigmoid_prob) * 1)]
end

den_gradient (generic function with 1 method)

In [45]:
den_gradient(x, y, 3, 0.4)

2-element Array{Float64,1}:
 -11124.344263615252
 144873.83418491355 

Hessian

$$
% <![CDATA[
H_{\ell(\hat{\theta})} = \begin{bmatrix}\begin{split}
\sum_{i=1}^{n}h_{\theta}(x_{i})(1-h_{\theta}(x_{i}))\theta_{1}\theta_{1},\ &
\sum_{i=1}^{n}h_{\theta}(x_{i})(1-h_{\theta}(x_{i}))\theta_{1}\\
\sum_{i=1}^{n}h_{\theta}(x_{i})(1-h_{\theta}(x_{i}))\theta_{1},\ &
\sum_{i=1}^{n}h_{\theta}(x_{i})(1-h_{\theta}(x_{i}))
\end{split}\end{bmatrix} %]]>
$$

In [48]:
function den_hessian(x, y, θ1, θ2)
    sigmoid_prob = den_sigmoid(x, θ1, θ2)
    a = sum(sigmoid_prob .* (1 .- sigmoid_prob) .* θ1 .* θ2)
    b = sum(sigmoid_prob .* (1 .- sigmoid_prob) .* θ1)
    c = sum(sigmoid_prob .* (1 .- sigmoid_prob))
    H = [a b; b c]
end

den_hessian (generic function with 1 method)

In [49]:
den_hessian(x, y, 3, 0.4)

2×2 Array{Float64,2}:
 1024.36  2560.9  
 2560.9    853.633

Newton's method algorithm: 

1. Find the tangent line to $f(x)$ at point $(x_n,y_n)$
    
$$
y=f'(x_n)(x−x_n)+f(x_n) 
$$
 
2. Find the x-intercept of the tangent line, $x_{n+1}$

$$
x_{n+1}=x_n−f(x_n)f′(x_n)
$$

3. Find the $y$ value at the x-intercept.

$$
yn+1=f(xn+1)
$$

4. If $y_{n+1}−y_n \approx 0$
return $y_{n+1}$ because we’ve converged!

5. Else update point $(x_n,y_n)$, and iterate

$$
x=x_{n+1} \\
y=y_{n+1}
$$

In [57]:
function den_newtons_method(x, y)
    # Initialize log_likelihood & parameters                                                                   
    θ1 = 15
    θ2 = -0.4 # The intercept term                                                                 
    Δl = Inf
    l = den_log_likelihood(x, y, θ1, θ2)   
    
    # Convergence Conditions                                                        
    δ = .0000000001                                                               
    max_iterations = 15                                                            
    i = 0         
    
    while abs(Δl) > δ & i < max_iterations
        i += 1
        gradient = den_gradient(x, y, θ1, θ2)
        hessian = den_hessian(x, y, θ1, θ2)
        H_inv = inv(hessian)
        
        delta = H_inv'transpose(gradient)
        d_θ1 = delta[1] 
        d_θ2 = delta[2]
        
        # Update
        θ1 += d_θ1
        θ2 += d_θ2
        
        # Update the log-likelihood at each iteration                                     
        l_new = den_log_likelihood(x, y, θ1, θ2)   
        Δl = l - l_new                                                           
        l = l_new 
        
    end
    [θ1 θ2]
end

den_newtons_method (generic function with 1 method)

In [58]:
den_newtons_method(x, y)

MethodError: MethodError: no method matching &(::Float64, ::Int64)
Closest candidates are:
  &(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  &(!Matched::Missing, ::Integer) at missing.jl:126
  &(!Matched::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}, ::T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:297
  ...